In [1]:
from google.colab import drive
drive.mount('/content/drive/')

PATH = '/content/drive/My Drive/Colab Notebooks/SOYO/Election/'

Mounted at /content/drive/


In [2]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install -q pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     |████████████████████████████████| 43 kB 1.7 MB/s 


In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from datetime import timedelta

In [4]:
from IPython.core.display import display, HTML 
display(HTML("<style>.contaeiner { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 100)

# 데이터 설명
* naver_df: ~3/3까지 데이터 전체
* google_df1: ~1/15까지 데이터(7일 단위)
* google_df2: 1/23 ~ 3/3까지 데이터(1일 단위)

### 주의!
* 구글 데이터 경우 1/15 ~ 1/22까지가 없음
* 1/9 ~ 1/15까지는 이재명, 윤석열 데이터밖에 없음

In [5]:
naver_df = pd.read_csv(PATH + 'dataset/naver_total.csv')
google_df1 = pd.read_csv(PATH + 'dataset/google_total_0127.csv')
google_df2 = pd.read_csv(PATH + 'dataset/google_total_0303.csv')

naver_df = naver_df.rename(columns={'Unnamed: 0': 'election_id'})
google_df1 = google_df1.rename(columns={'Unnamed: 0': 'election_id'})
google_df2 = google_df2.rename(columns={'Unnamed: 0': 'election_id'})

naver_df = naver_df.add_prefix('naver_')
google_df1 = google_df1.add_prefix('google_')
google_df2 = google_df2.add_prefix('google_')

print(naver_df.shape)
print(google_df1.shape)
print(google_df2.shape)

(253, 322)
(250, 261)
(254, 7)


In [6]:
google_df1 = google_df1.fillna(0)
google_df1.drop(['google_2022-01-09_2022-01-15'], axis=1, inplace=True) # 해당 날짜에 해당하는 컬럼 삭제(이재명 vs 윤석열만 있음)
google_df1.head()

,google_election_id,google_2017-01-22_2017-01-28,google_2017-01-29_2017-02-04,google_2017-02-05_2017-02-11,google_2017-02-12_2017-02-18,google_2017-02-19_2017-02-25,google_2017-02-26_2017-03-04,google_2017-03-05_2017-03-11,google_2017-03-12_2017-03-18,google_2017-03-19_2017-03-25,google_2017-03-26_2017-04-01,google_2017-04-02_2017-04-08,google_2017-04-09_2017-04-15,google_2017-04-16_2017-04-22,google_2017-04-23_2017-04-29,google_2017-04-30_2017-05-06,google_2017-05-07_2017-05-13,google_2017-05-14_2017-05-20,google_2017-05-21_2017-05-27,google_2017-05-28_2017-06-03,google_2017-06-04_2017-06-10,google_2017-06-11_2017-06-17,google_2017-06-18_2017-06-24,google_2017-06-25_2017-07-01,google_2017-07-02_2017-07-08,google_2017-07-09_2017-07-15,google_2017-07-16_2017-07-22,google_2017-07-23_2017-07-29,google_2017-07-30_2017-08-05,google_2017-08-06_2017-08-12,google_2017-08-13_2017-08-19,google_2017-08-20_2017-08-26,google_2017-08-27_2017-09-02,google_2017-09-03_2017-09-09,google_2017-09-10_2017-09-16,google_2017-09-17_2017-09-23,google_2017-09-24_2017-09-30,google_2017-10-01_2017-10-07,google_2017-10-08_2017-10-14,google_2017-10-15_2017-10-21,google_2017-10-22_2017-10-28,google_2017-10-29_2017-11-04,google_2017-11-05_2017-11-11,google_2017-11-12_2017-11-18,google_2017-11-19_2017-11-25,google_2017-11-26_2017-12-02,google_2017-12-03_2017-12-09,google_2017-12-10_2017-12-16,google_2017-12-17_2017-12-23,google_2017-12-24_2017-12-30,...,google_2021-01-24_2021-01-30,google_2021-01-31_2021-02-06,google_2021-02-07_2021-02-13,google_2021-02-14_2021-02-20,google_2021-02-21_2021-02-27,google_2021-02-28_2021-03-06,google_2021-03-07_2021-03-13,google_2021-03-14_2021-03-20,google_2021-03-21_2021-03-27,google_2021-03-28_2021-04-03,google_2021-04-04_2021-04-10,google_2021-04-11_2021-04-17,google_2021-04-18_2021-04-24,google_2021-04-25_2021-05-01,google_2021-05-02_2021-05-08,google_2021-05-09_2021-05-15,google_2021-05-16_2021-05-22,google_2021-05-23_2021-05-29,google_2021-05-30_2021-06-05,google_2021-06-06_2021-06-12,google_2021-06-13_2021-06-19,google_2021-06-20_2021-06-26,google_2021-06-27_2021-07-03,google_2021-07-04_2021-07-10,google_2021-07-11_2021-07-17,google_2021-07-18_2021-07-24,google_2021-07-25_2021-07-31,google_2021-08-01_2021-08-07,google_2021-08-08_2021-08-14,google_2021-08-15_2021-08-21,google_2021-08-22_2021-08-28,google_2021-08-29_2021-09-04,google_2021-09-05_2021-09-11,google_2021-09-12_2021-09-18,google_2021-09-19_2021-09-25,google_2021-09-26_2021-10-02,google_2021-10-03_2021-10-09,google_2021-10-10_2021-10-16,google_2021-10-17_2021-10-23,google_2021-10-24_2021-10-30,google_2021-10-31_2021-11-06,google_2021-11-07_2021-11-13,google_2021-11-14_2021-11-20,google_2021-11-21_2021-11-27,google_2021-11-28_2021-12-04,google_2021-12-05_2021-12-11,google_2021-12-12_2021-12-18,google_2021-12-19_2021-12-25,google_2021-12-26_2022-01-01,google_2022-01-02_2022-01-08
0,512,-48.0,24.0,5.0,9.0,1.0,7.0,-17.0,30.0,-3.0,0.0,3.0,-7.0,14.0,-7.0,68.0,-58.0,38.0,-22.0,-22.0,7.0,-4.0,0.0,0.0,-4.0,2.0,4.0,-4.0,4.0,-2.0,-2.0,2.0,-2.0,12.0,-5.0,-5.0,4.0,-6.0,0.0,4.0,2.0,-5.0,1.0,0.0,-4.0,6.0,-2.0,-3.0,1.0,4.0,...,6.0,-8.0,5.0,-3.0,20.0,-14.0,-8.0,-7.0,9.0,2.0,-8.0,2.0,-4.0,3.0,-1.0,11.0,-8.0,-2.0,4.0,1.0,6.0,-4.0,10.0,13.0,13.0,-9.0,-9.0,-6.0,-2.0,-4.0,3.0,5.0,-8.0,8.0,-4.0,0.0,40.0,-50.0,-1.0,0.0,4.0,-10.0,5.0,0.0,1.0,-3.0,5.0,-5.0,0.0,3.0
1,513,-4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,-4.0,0.0,0.0,0.0,0.0,...,0.0,-3.0,3.0,0.0,0.0,-3.0,0.0,3.0,0.0,0.0,0.0,-3.0,0.0,3.0,-6.0,6.0,-6.0,6.0,0.0,-3.0,-3.0,3.0,-4.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,-100.0,97.0
2,514,0.0,0.0,0.0,0.0,17.0,-17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-36.0,36.0,0.0,-19.0,56.0,-37.0,0.0,0.0,0.0,0.0,0.0,38.0,-38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-36.0,36.0,0.0,0.0,18.0,-18.0,0.0,0.0,0.0,0.0,18.0,-18.0,1

In [11]:
print(set(google_df2['google_election_id']).difference(set(naver_df['naver_election_id'])))
print(set(google_df2['google_election_id']).difference(set(google_df1['google_election_id'])))

{578}
{577, 578, 657, 667}


In [12]:
# google_df2 기준으로 후보쌍 정리 -> {577, 578, 657, 667} 제거
candi_list = [577, 578, 657, 667]
naver_df = naver_df.loc[~naver_df['naver_election_id'].isin(candi_list)]
google_df1 = google_df1.loc[~google_df1['google_election_id'].isin(candi_list)]
google_df2 = google_df2.loc[~google_df2['google_election_id'].isin(candi_list)]

print(naver_df.shape, google_df1.shape, google_df2.shape)

(250, 322) (250, 260) (250, 7)


In [13]:
google_df = pd.merge(google_df1, google_df2, how='inner', on='google_election_id')

In [14]:
import pymysql

host = '****.****.ap-northeast-2.rds.amazonaws.com'
port = 3306
username = '****'
password = '****'
db = '****'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

In [15]:
sql_state = 'SELECT * FROM election_history'

history_df = pd.read_sql_query(sql_state, conn)
conn.close()

In [16]:
history_df['당1'] = history_df['candi_1'].apply(lambda x: x.split('_')[1])
history_df['당2'] = history_df['candi_2'].apply(lambda x: x.split('_')[1])
history1 = history_df.loc[(history_df['당1'].isin(['더불어민주당', '미래통합당']))]
history = history1.loc[(history1['당2'].isin(['더불어민주당', '미래통합당']))]
history = pd.concat([history_df.loc[history_df['id'] == 1], history])
history

,id,election,candi_1,candi_2,elected,region,당1,당2
0,1,20_president,이재명_더불어민주당,윤석열_국민의힘,-1,,더불어민주당,국민의힘
1,512,21_national_assembly,이낙연_더불어민주당,황교안_미래통합당,0,서울\r,더불어민주당,미래통합당
2,513,21_national_assembly,강태웅_더불어민주당,권영세_미래통합당,1,서울\r,더불어민주당,미래통합당
3,514,21_national_assembly,홍익표_더불어민주당,진수희_미래통합당,0,서울\r,더불어민주당,미래통합당
4,515,21_national_assembly,박성준_더불어민주당,지상욱_미래통합당,0,서울\r,더불어민주당,미래통합당
...,...,...,...,...,...,...,...,...
248,759,21_national_assembly,이재영_더불어민주당,윤영석_미래통합당,1,경남\r,더불어민주당,미래통합당
249,760,21_national_assembly,김두관_더불어민주당,나동연_미래통합당,0,경남\r,더불어민주당,미래통합당
251,762,21_national_assembly,송재호_더불어민주당,장성철_미래통합당,0,제주\r,더불어민주당,미래통합당
252,763,21_national_assembly,오영훈_더불어민주당,부상일_미래통합당,0,제주\r,더불어민주당,미래통합당


In [17]:
from sklearn.model_selection import KFold, train_test_split

import xgboost as xgb
import lightgbm as lgb

# metric
from sklearn.metrics import f1_score, roc_auc_score

In [18]:
SEED = 42

n_splits = 5
cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

# 날짜별로 나누기
* 1차: naver -> ~1/27까지, google -> ~1/8까지
* 2차: naver -> ~2/3까지, google -> ~1/29까지
* 3차: naver -> ~2/10까지, google -> ~2/5까지
* 4차: naver -> ~2/17까지, google -> ~2/12까지
* 5차: naver -> ~2/24까지, google -> ~2/19까지
* 6차: naver -> ~3/3까지, google -> ~2/26까지

In [22]:
naver1 = naver_df.iloc[:, :317]
naver2 = naver_df.iloc[:, [0, 317]]
naver3 = naver_df.iloc[:, [0, 318]]
naver4 = naver_df.iloc[:, [0, 319]]
naver5 = naver_df.iloc[:, [0, 320]]
naver6 = naver_df.iloc[:, [0, 321]]

google1 = google_df.iloc[:, :260]
google2 = google_df.iloc[:, [0, 260]]
google3 = google_df.iloc[:, [0, 261]]
google4 = google_df.iloc[:, [0, 262]]
google5 = google_df.iloc[:, [0, 263]]
google6 = google_df.iloc[:, [0, 264]]

In [23]:
# 네이버 + 구글 merge
merged_df = pd.merge(naver1, google1, how='inner', left_on='naver_election_id', right_on='google_election_id')

# target 값 추가한 최종 데이터프레임
df = pd.merge(merged_df, history[['id', 'elected']], how='inner', left_on='naver_election_id', right_on='id')
df.drop(['naver_election_id', 'google_election_id'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [25]:
merged2 = pd.merge(df, naver2, how='inner', left_on='id', right_on='naver_election_id')
merged2 = pd.merge(merged2, google2, how='inner', left_on='id', right_on='google_election_id')
merged2.drop(['naver_election_id', 'google_election_id'], axis=1, inplace=True)

merged3 = pd.merge(merged2, naver3, how='inner', left_on='id', right_on='naver_election_id')
merged3 = pd.merge(merged3, google2, how='inner', left_on='id', right_on='google_election_id')
merged3.drop(['naver_election_id', 'google_election_id'], axis=1, inplace=True)

merged4 = pd.merge(merged3, naver4, how='inner', left_on='id', right_on='naver_election_id')
merged4 = pd.merge(merged4, google4, how='inner', left_on='id', right_on='google_election_id')
merged4.drop(['naver_election_id', 'google_election_id'], axis=1, inplace=True)

merged5 = pd.merge(merged4, naver5, how='inner', left_on='id', right_on='naver_election_id')
merged5 = pd.merge(merged5, google5, how='inner', left_on='id', right_on='google_election_id')
merged5.drop(['naver_election_id', 'google_election_id'], axis=1, inplace=True)

merged6 = pd.merge(merged5, naver6, how='inner', left_on='id', right_on='naver_election_id')
merged6 = pd.merge(merged6, google6, how='inner', left_on='id', right_on='google_election_id')
merged6.drop(['naver_election_id', 'google_election_id'], axis=1, inplace=True)

print(merged6.shape)

(217, 587)


In [26]:
# 총 217개 후보쌍 존재
df['elected'].value_counts()

 0    133
 1     83
-1      1
Name: elected, dtype: int64

In [27]:
# 학습셋, 테스트셋 분리
test1 = df.loc[df['id'] == 1]
test1.drop(['id', 'elected'], axis=1, inplace=True)
train1 = df.loc[df['id'] != 1]

test2 = merged2.loc[merged2['id'] == 1]
test2.drop(['id', 'elected'], axis=1, inplace=True)
train2 = merged2.loc[merged2['id'] != 1]

test3 = merged3.loc[merged3['id'] == 1]
test3.drop(['id', 'elected'], axis=1, inplace=True)
train3 = merged3.loc[merged3['id'] != 1]

test4 = merged4.loc[merged4['id'] == 1]
test4.drop(['id', 'elected'], axis=1, inplace=True)
train4 = merged4.loc[merged4['id'] != 1]

test5 = merged5.loc[merged5['id'] == 1]
test5.drop(['id', 'elected'], axis=1, inplace=True)
train5 = merged5.loc[merged5['id'] != 1]

test6 = merged6.loc[merged6['id'] == 1]
test6.drop(['id', 'elected'], axis=1, inplace=True)
train6 = merged6.loc[merged6['id'] != 1]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
X1 = train1.drop(['id', 'elected'], axis=1)
X1.reset_index(drop=True, inplace=True)
y1 = train1['elected']
y1.reset_index(drop=True, inplace=True)

X2 = train2.drop(['id', 'elected'], axis=1)
X2.reset_index(drop=True, inplace=True)
y2 = train2['elected']
y2.reset_index(drop=True, inplace=True)

X3 = train3.drop(['id', 'elected'], axis=1)
X3.reset_index(drop=True, inplace=True)
y3 = train3['elected']
y3.reset_index(drop=True, inplace=True)

X4 = train4.drop(['id', 'elected'], axis=1)
X4.reset_index(drop=True, inplace=True)
y4 = train4['elected']
y4.reset_index(drop=True, inplace=True)

X5 = train5.drop(['id', 'elected'], axis=1)
X5.reset_index(drop=True, inplace=True)
y5 = train5['elected']
y5.reset_index(drop=True, inplace=True)

X6 = train6.drop(['id', 'elected'], axis=1)
X6.reset_index(drop=True, inplace=True)
y6 = train6['elected']
y6.reset_index(drop=True, inplace=True)

# XGB

In [29]:
def xgb_model(X, y):
  params = {
    'objective': 'binary:logistic', 
    'learning_rate': 0.03,
    'eval_metric': 'auc',
    # "tree_method": "gpu_hist",
    'seed': 42
  }

  xgb_pred_list = []
  xgb_models = []
  xgb_score = []

  for train_index, val_index in cv.split(X):
    train_features = X.loc[train_index]
    train_target = y.loc[train_index]
    
    val_features = X.loc[val_index]
    val_target = y.loc[val_index]
    
    d_training = xgb.DMatrix(train_features, label=train_target)
    d_val = xgb.DMatrix(val_features, label=val_target)
    wlist = [(d_training, 'train'), (d_val,'eval')]

    xgb_model = xgb.train(params, d_training, evals=wlist, num_boost_round=2000, verbose_eval=100, early_stopping_rounds=50)
    xgb_models.append(xgb_model)

    val_pred = xgb_model.predict(xgb.DMatrix(val_features))
    xgb_roc_score = roc_auc_score(val_target, val_pred, average='macro')
    
    xgb_score.append(xgb_roc_score)

  return xgb_score, xgb_models

In [30]:
xgb_score1, xgb_models1 = xgb_model(X1, y1)
xgb_score2, xgb_models2 = xgb_model(X2, y2)
xgb_score3, xgb_models3 = xgb_model(X3, y3)
xgb_score4, xgb_models4 = xgb_model(X4, y4)
xgb_score5, xgb_models5 = xgb_model(X5, y5)
xgb_score6, xgb_models6 = xgb_model(X6, y6)

print(np.mean(xgb_score1), np.mean(xgb_score2), np.mean(xgb_score3), np.mean(xgb_score4), np.mean(xgb_score5), np.mean(xgb_score6))

[0]	train-auc:0.996182	eval-auc:0.770115
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[100]	train-auc:1	eval-auc:0.822989
[200]	train-auc:1	eval-auc:0.845977
Stopping. Best iteration:
[166]	train-auc:1	eval-auc:0.848276

[0]	train-auc:0.986761	eval-auc:0.87931
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[100]	train-auc:1	eval-auc:0.913793
Stopping. Best iteration:
[52]	train-auc:1	eval-auc:0.921182

[0]	train-auc:0.98794	eval-auc:0.801948
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[100]	train-auc:1	eval-auc:0.839827
Stopping. Best iteration:
[82]	train-auc:1	eval-auc:0.841991

[0]	train-auc:0.988176	eval-auc:0.734163
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.


In [31]:
def pred_test(xgb_models, test):
  test_pred_list = []
  for i in range(n_splits):
    test_pred = xgb_models[i].predict(xgb.DMatrix(test))[0] # 1이 될 확률 리턴(array 형태기 때문에 [0]을 통해 숫자만 추출)
    test_pred_list.append(test_pred)

  test_mean = np.mean(test_pred_list)
  if test_mean > 0.5:
    print(f'{np.round(test_mean, 2) * 100}%의 확률로 윤석열 승리!')
  else:
    print(f'{np.round(1 - test_mean, 2) * 100}%의 확률로 이재명 승리!')

In [32]:
pred_test(xgb_models1, test1)
print('\n')
pred_test(xgb_models2, test2)
print('\n')
pred_test(xgb_models3, test3)
print('\n')
pred_test(xgb_models4, test4)
print('\n')
pred_test(xgb_models5, test5)
print('\n')
pred_test(xgb_models6, test6)

56.00000000000001%의 확률로 이재명 승리!


56.00000000000001%의 확률로 이재명 승리!


57.99999999999999%의 확률로 이재명 승리!


60.0%의 확률로 이재명 승리!


57.99999999999999%의 확률로 이재명 승리!


59.0%의 확률로 이재명 승리!


# LGBM

In [33]:
def lgbm_model(X, y):
  params = {
    'objective': 'binary', 
    'learning_rate': 0.03,
    'metric': 'auc',
    # "tree_method": "gpu_hist",
    'seed': 2021
  }

  lgb_pred_list = []
  lgb_models = []
  lgb_score = []

  for train_index, val_index in cv.split(X):
    train_features = X.loc[train_index]
    train_target = y.loc[train_index]
    
    val_features = X.loc[val_index]
    val_target = y.loc[val_index]
    
    d_training = lgb.Dataset(train_features, label=train_target, free_raw_data=False)
    d_val = lgb.Dataset(val_features, label=val_target, free_raw_data=False)

    lgb_model = lgb.train(params, train_set=d_training, valid_sets=d_val, num_boost_round=2000, verbose_eval=100, early_stopping_rounds=50)
    lgb_models.append(lgb_model)

    val_pred = lgb_model.predict(val_features)
    lgb_roc_score = roc_auc_score(val_target, val_pred, average='macro')
    
    lgb_score.append(lgb_roc_score)

  return lgb_score, lgb_models

In [34]:
lgb_score1, lgb_models1 = lgbm_model(X1, y1)
lgb_score2, lgb_models2 = lgbm_model(X2, y2)
lgb_score3, lgb_models3 = lgbm_model(X3, y3)
lgb_score4, lgb_models4 = lgbm_model(X4, y4)
lgb_score5, lgb_models5 = lgbm_model(X5, y5)
lgb_score6, lgb_models6 = lgbm_model(X6, y6)

print(np.mean(lgb_score1), np.mean(lgb_score2), np.mean(lgb_score3), np.mean(lgb_score4), np.mean(lgb_score5), np.mean(lgb_score6))

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[9]	valid_0's auc: 0.811494
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[26]	valid_0's auc: 0.921182
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[5]	valid_0's auc: 0.853896
Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.821267
Early stopping, best iteration is:
[68]	valid_0's auc: 0.837104
Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.888889
Early stopping, best iteration is:
[96]	valid_0's auc: 0.893519
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[9]	valid_0's auc: 0.811494
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[26]	valid_0's auc: 0.921182
Training until validation scores don't improve for 50 rounds.

In [35]:
def lgb_pred_test(lgb_models, test):
  test_pred_list = []
  for i in range(n_splits):
    test_pred = lgb_models[i].predict(test)[0]
    test_pred_list.append(test_pred)

  test_mean = np.mean(test_pred_list)
  if test_mean > 0.5:
    print(f'{np.round(test_mean, 2) * 100}%의 확률로 윤석열 승리!')
  else:
    print(f'{np.round(1 - test_mean, 2) * 100}%의 확률로 이재명 승리!')

In [36]:
lgb_pred_test(lgb_models1, test1)
print('\n')
lgb_pred_test(lgb_models2, test2)
print('\n')
lgb_pred_test(lgb_models3, test3)
print('\n')
lgb_pred_test(lgb_models4, test4)
print('\n')
lgb_pred_test(lgb_models5, test5)
print('\n')
lgb_pred_test(lgb_models6, test6)

51.0%의 확률로 이재명 승리!


51.0%의 확률로 이재명 승리!


51.0%의 확률로 이재명 승리!


51.0%의 확률로 이재명 승리!


51.0%의 확률로 이재명 승리!


51.0%의 확률로 이재명 승리!


# 결론
* LGBM의 Cross Validation 성능이 더 높게 나오긴 하나, 이번엔 모두 51% 확률로 동일하게 나옴
* XGB의 결과를 사용하는 것이 나아보임